# Programmation GPU - Projet #

In [ ]:
import numpy as np
from numba import cuda, uint32
import time

## Kernel CUDA pour FNV-1a ##

In [ ]:
@cuda.jit
def fnv1a_kernel(messages, lengths, output):
    FNV_OFFSET_BASIS = uint32(0x811c9dc5)
    FNV_PRIME = uint32(0x01000193)

    idx = cuda.grid(1)
    if idx >= messages.shape[0]:
        return

    h = FNV_OFFSET_BASIS
    for i in range(lengths[idx]):
        h ^= messages[idx, i]
        h = uint32(h * FNV_PRIME)  # ✅ FIX ICI

    output[idx] = h

## Fonction d’appel CPU → GPU ##

In [ ]:
def hash_all_gpu(input_list):
    """
    Hash une liste de chaînes avec FNV-1a sur GPU (Numba CUDA)
    """
    print(f"Hash de {len(input_list):,} chaînes en GPU (FNV-1a)...")

    n = len(input_list)
    max_len = 32  # on suppose que les chaînes font au max 32 caractères

    # Préparation des données : tableau 2D de uint8
    messages_np = np.zeros((n, max_len), dtype=np.uint8)
    lengths_np = np.zeros(n, dtype=np.uint8)

    for i, s in enumerate(input_list):
        encoded = s.encode('utf-8')[:max_len]
        messages_np[i, :len(encoded)] = list(encoded)
        lengths_np[i] = len(encoded)

    output_np = np.zeros(n, dtype=np.uint32)

    # Allocation et copie sur GPU
    d_messages = cuda.to_device(messages_np)
    d_lengths = cuda.to_device(lengths_np)
    d_output = cuda.device_array(n, dtype=np.uint32)

    threads_per_block = 128
    blocks_per_grid = (n + threads_per_block - 1) // threads_per_block

    # Lancement du kernel
    start = time.time()
    fnv1a_kernel[blocks_per_grid, threads_per_block](d_messages, d_lengths, d_output)
    cuda.synchronize()
    end = time.time()

    # Récupération des résultats
    d_output.copy_to_host(ary=output_np)

    print(f"Temps total : {end - start:.4f} secondes")
    print(f"Exemple : {input_list[0]} -> {output_np[0]:08x}")
    return output_np

## Test principal ##

In [ ]:
if __name__ == "__main__":
    test_data = [f"data_{i}" for i in range(1_000_000)]  # 1 million
    hashes_gpu = hash_all_gpu(test_data)